In [1]:
import numpy as np
import random
import optuna
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"

In [3]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    Returns only the F1-score for trend change.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [5]:
# Cargar datos
train = pd.read_csv("../../../data/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values

In [7]:
def objective(trial):
    set_seeds(SEED)
    n_estimators = trial.suggest_int("n_estimators", 50, 400)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=SEED
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = trend_changes_true(y_val, y_pred)
    return score

In [ ]:
set_seeds(SEED)
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=40)

[I 2025-08-17 22:38:24,659] A new study created in memory with name: no-name-2db7d437-0114-45b0-94ed-38888005113a
[I 2025-08-17 22:38:27,924] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 117, 'learning_rate': 0.06447363728960653, 'max_depth': 5, 'subsample': 0.9141434334855076, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.0.
[I 2025-08-17 22:38:32,126] Trial 1 finished with value: 0.041666666666666664 and parameters: {'n_estimators': 147, 'learning_rate': 0.11047391773518671, 'max_depth': 10, 'subsample': 0.9503730538968379, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.041666666666666664.
[I 2025-08-17 22:38:39,406] Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 289, 'learning_rate': 0.08457591696604633, 'max_depth': 5, 'subsample': 0.82447847442625, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.041666666666666664.
[I 2025-08-17 22:38:47,628] Trial 3

Mejores hiperparámetros encontrados:
{'n_estimators': 282, 'learning_rate': 0.036270679129652636, 'max_depth': 4, 'subsample': 0.6426763526875084, 'min_samples_split': 10, 'min_samples_leaf': 10}
Mejor score de trend change: 0.3000


In [9]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")

Mejores hiperparámetros encontrados:
{'n_estimators': 282, 'learning_rate': 0.036270679129652636, 'max_depth': 4, 'subsample': 0.6426763526875084, 'min_samples_split': 10, 'min_samples_leaf': 10}
Mejor score de f1-score: 0.3000


In [10]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
best_params = study.best_params
final_model = GradientBoostingClassifier(
    n_estimators=best_params["n_estimators"],
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    subsample=best_params["subsample"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    random_state=SEED
)
final_model.fit(X_train, y_train)
y_pred_final = final_model.predict(X_val)
print("GradientBoostingClassifier FINAL trend_changes_score:\n", trend_changes_score(y_val, y_pred_final))

GradientBoostingClassifier FINAL trend_changes_score:
               precision    recall  f1-score   support

       False     0.8962    0.8482    0.8716       112
        True     0.2609    0.3529    0.3000        17

    accuracy                         0.7829       129
   macro avg     0.5785    0.6006    0.5858       129
weighted avg     0.8125    0.7829    0.7962       129

